In [ ]:
from time_domain_pid import (
    SecondOrderDelay, 
    NoController, 
    DiscretePIDController, 
    TimeDomainLoopSimulator,
)
from interfaces import InitialConditions
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
plant = SecondOrderDelay(
        delta=500e-9,
        w0=500e3,
        w1=1000e3,
        gain=0.5,
    )

In [ ]:
# Check that the plant has a reasonable impulse response
plt.figure()
t = plant.impulse_t
plt.plot(t, plant.impulse_response(t))
print(f"requested gain = {plant.gain}, actual gain = {np.sum(plant.impulse_response(t))*plant.dt}")

In [ ]:
# Setup a segmented 
t = np.linspace(-1e-6, 500e-6, 1000)
seg1 = 0.5
seg2 = 0.1
seg3 = lambda t1, t2, y1, y2: (t-t1)*(y2-y1)/(t2-t1) + y1
r = seg1*(t > 0)*(t < 100e-6)+ \
    seg2*(t>= 100e-6)*(t < 200e-6)+ \
    seg3(200e-6, 500e-6, 0.1, 0.5)*(t>=200e-6)
start_ind = np.argmin(np.abs(t))
init_conds = InitialConditions(
    t = t,
    r = r,
    active = np.ones(t.shape, dtype = np.bool),
    initial_u = 0.0,
)

In [ ]:
# Calculate PID
no_pid_controller = NoController(gain=2.0)
i_controller = DiscretePIDController(kp=1.0, ki=200e4, kd=2e-7)

history_none = TimeDomainLoopSimulator(plant, no_pid_controller).simulate(init_conds)
history_i = TimeDomainLoopSimulator(plant, i_controller).simulate(init_conds)

In [ ]:
# Display results
plt.figure()
plt.plot(init_conds.t, init_conds.r, label='r, setpoint')
plt.plot(history_none.t, history_none.y, label='y, no controller')
plt.plot(history_i.t, history_i.y, label='y, pid controller')
plt.legend()
plt.xlabel('time (s)')
plt.ylabel('process value (V)')